In [16]:
import import_ipynb
from dijkstra import nodes, arc_matrix, get_shortest_path, dijkstra, format_path

from pprint import pprint
import numpy as np


In [17]:
def init_tamponi_per_reparto():
    tamponi_per_reparto = np.array([0, 0, 0, 3, 0, 0, 0, 6, 0, 0, 0, 0, 12, 0, 0, 0, 7, 0, 0, 0, 10, 0,
        0, 0, 3, 0, 30, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]).astype(int)

    return tamponi_per_reparto

In [18]:
tamponi_per_reparto = init_tamponi_per_reparto()
print(tamponi_per_reparto)

[ 0  0  0  3  0  0  0  6  0  0  0  0 12  0  0  0  7  0  0  0 10  0  0  0
  3  0 30  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]


In [19]:
def estrai_reparti_con_tamponi(tamponi_per_reparto):
    return [i for i,t in enumerate(tamponi_per_reparto) if t > 0]

In [20]:
reparti_con_tamponi = estrai_reparti_con_tamponi(tamponi_per_reparto)

print(reparti_con_tamponi)

for index,node in enumerate(reparti_con_tamponi):
    print(f"{index}. Reparto con indice {node} e nome '{nodes[node]}' ha {tamponi_per_reparto[node]} tamponi")

[3, 7, 12, 16, 20, 24, 26, 28]
0. Reparto con indice 3 e nome '4' ha 3 tamponi
1. Reparto con indice 7 e nome '7' ha 6 tamponi
2. Reparto con indice 12 e nome '11n' ha 12 tamponi
3. Reparto con indice 16 e nome '14' ha 7 tamponi
4. Reparto con indice 20 e nome '19' ha 10 tamponi
5. Reparto con indice 24 e nome '22' ha 3 tamponi
6. Reparto con indice 26 e nome '24' ha 30 tamponi
7. Reparto con indice 28 e nome '25/2' ha 4 tamponi


In [21]:
# tamponi / minuto
def compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto):
    return compute_density_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto)

def compute_tamponi_in_nodo(p, prev, tamponi_per_reparto, tamponi_per_path, start_node=None):
    if prev is None and start_node is not None:
        _, prev = np.array(dijkstra(arc_matrix, start_node))
        tamponi_per_path = -1 * np.ones(len(prev))
    
    path = []
    path.append(p)

    prev = np.array(prev).astype(int)
    while p != -1:
        p = prev[p]
        path.append(p)

    path = path[::-1] # reverse
    path = path[1:] # remove start node

    temp_tamponi = 0
    for p in path:
        if tamponi_per_path[p] == -1:
            # print(p,"aggiungo questo reparto al dizionario")
            temp_tamponi += tamponi_per_reparto[p]
            tamponi_per_path[p] = temp_tamponi
        else:
            temp_tamponi = tamponi_per_path[p]
        
        # print(p, nodes[p], tamponi_per_reparto[p], temp_tamponi)
    
    return tamponi_per_path
        

def compute_score_row(arc_matrix, reparti_con_tamponi, indice_reparto):
    time, prev = np.array(dijkstra(arc_matrix, indice_reparto))
    # print(prev)
    
    tamponi_per_path = -1 * np.ones(len(prev))
    for p in reparti_con_tamponi:
        tamponi_per_path = compute_tamponi_in_nodo(p, prev, tamponi_per_reparto, tamponi_per_path)
    
    time = time[reparti_con_tamponi]
    tamponi_per_path = tamponi_per_path[reparti_con_tamponi]
    
    # print(tamponi_per_path)
    density_score = np.divide(tamponi_per_path, time, out=-np.inf*np.ones_like(time), where=time!=0)
    return density_score

def compute_density_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto):
    reparti_con_tamponi = np.array(reparti_con_tamponi)
    score_matrix = np.zeros((len(reparti_con_tamponi), len(reparti_con_tamponi)))
    for i, indice_reparto in enumerate(reparti_con_tamponi):
        score_matrix[i] = compute_score_row(arc_matrix, reparti_con_tamponi, indice_reparto)
    return score_matrix

In [22]:
def retrieve_best_score(score_matrix, index=0):
    return np.argmax(score_matrix[index]), np.max(score_matrix[index])

In [23]:
score_matrix = compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto)
for index,node in enumerate(reparti_con_tamponi):
    best_index, best_score = retrieve_best_score(score_matrix, index)
    print(f"{index}. Reparto con indice {node} e nome '{nodes[node]}' ha best score in reparto {best_index}. indice {reparti_con_tamponi[best_index]} con nome '{nodes[reparti_con_tamponi[best_index]]}' con score {best_score}")

0. Reparto con indice 3 e nome '4' ha best score in reparto 6. indice 26 con nome '24' con score 2.2941176470588234
1. Reparto con indice 7 e nome '7' ha best score in reparto 6. indice 26 con nome '24' con score 7.2
2. Reparto con indice 12 e nome '11n' ha best score in reparto 6. indice 26 con nome '24' con score 3.2
3. Reparto con indice 16 e nome '14' ha best score in reparto 6. indice 26 con nome '24' con score 2.5294117647058822
4. Reparto con indice 20 e nome '19' ha best score in reparto 6. indice 26 con nome '24' con score 2.5
5. Reparto con indice 24 e nome '22' ha best score in reparto 6. indice 26 con nome '24' con score 3.3
6. Reparto con indice 26 e nome '24' ha best score in reparto 1. indice 7 con nome '7' con score 7.2
7. Reparto con indice 28 e nome '25/2' ha best score in reparto 6. indice 26 con nome '24' con score 1.4166666666666667


In [24]:
def get_index_from_node(node, reparti_con_tamponi):
    try:
        index = reparti_con_tamponi.index(node)
    except:
        # print("Nodo non trovato tra quelli con tamponi")
        score_row = compute_score_row(arc_matrix, reparti_con_tamponi, node)
        index, _ = retrieve_best_score(score_row)
        # print(score_row)
                
    return index

In [25]:
def get_next_node(start_node, reparti_con_tamponi, score_matrix, tamponi_per_reparto):
    index = get_index_from_node(start_node, reparti_con_tamponi)
    next_index, score = retrieve_best_score(score_matrix, index)
    path, _ = get_shortest_path(arc_matrix, start_node, reparti_con_tamponi[next_index])
    
    for p in path[1:]:
        if p in reparti_con_tamponi and tamponi_per_reparto[p] > 0:
            next_index = reparti_con_tamponi.index(p)
            break
    
    score_matrix[:,index] = -np.inf
    return reparti_con_tamponi[next_index], score

In [26]:
def stop_condition(residual_time, tamponi_trasportati, step):
    return residual_time <= 0 or tamponi_trasportati >= 96 or step >= len(reparti_con_tamponi) # aggiungere tempo al lab; numero di tamponi

In [27]:
def build_tsp_path(arc_matrix, reparti_con_tamponi, tamponi_per_reparto_orig, residual_time, tamponi_trasportati, current_node):
    tamponi_per_reparto = tamponi_per_reparto_orig.copy()
    path = []
    score_matrix = compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto)

    step = 1
    while not stop_condition(residual_time, tamponi_trasportati, step):
        next_node, score = get_next_node(current_node, reparti_con_tamponi, score_matrix, tamponi_per_reparto)
        distances, _ = dijkstra(arc_matrix, current_node)
        tempo_di_percorrenza = distances[next_node]
        residual_time -= tempo_di_percorrenza
        tamponi_per_path = compute_tamponi_in_nodo(next_node, None, tamponi_per_reparto, None, start_node=current_node)
        tamponi_trasportati += tamponi_per_path[next_node]
        print(f"{step}. Next node from '{nodes[current_node]}' is '{nodes[next_node]}' with time {tempo_di_percorrenza} and tamponi in path: {tamponi_per_path[next_node]}. Residual time is {residual_time}, Tamponi trasportati {tamponi_trasportati}. (score={score})")
        tamponi_per_reparto[next_node] = 0
        
        path.append(next_node)
        current_node = next_node
        step += 1
    
    return path

In [28]:
start_node = nodes.index('4')
residual_time = 120
tamponi_trasportati = 0

path = build_tsp_path(arc_matrix, reparti_con_tamponi, tamponi_per_reparto, residual_time, tamponi_trasportati, start_node)
print(f"Path: {format_path(path)}")

1. Next node from '4' is '7' with time 12.0 and tamponi in path: 9.0. Residual time is 108.0, Tamponi trasportati 9.0. (score=2.2941176470588234)
2. Next node from '7' is '24' with time 5.0 and tamponi in path: 30.0. Residual time is 103.0, Tamponi trasportati 39.0. (score=7.2)
3. Next node from '24' is '22' with time 10.0 and tamponi in path: 3.0. Residual time is 93.0, Tamponi trasportati 42.0. (score=3.3)
4. Next node from '22' is '19' with time 10.0 and tamponi in path: 10.0. Residual time is 83.0, Tamponi trasportati 52.0. (score=1.3)
5. Next node from '19' is '25/2' with time 14.0 and tamponi in path: 4.0. Residual time is 69.0, Tamponi trasportati 56.0. (score=1.0)
6. Next node from '25/2' is '11n' with time 37.0 and tamponi in path: 12.0. Residual time is 32.0, Tamponi trasportati 68.0. (score=0.5945945945945946)
7. Next node from '11n' is '14' with time 10.0 and tamponi in path: 7.0. Residual time is 22.0, Tamponi trasportati 75.0. (score=1.9)
Path: 7 -> 24 -> 22 -> 19 -> 25/2

In [29]:
# TODO
# matrice di score tra start_node e i reparti con tamponi: sembra OK
# matrice di score tra i reparti con tamponi e laboratorio + aggiungere nelle stop condition
# aggiungere tempo di percorrenza (corridoio) A -> B

start_node = nodes.index('X1')
residual_time = 120
tamponi_trasportati = 0
path = build_tsp_path(arc_matrix, reparti_con_tamponi, tamponi_per_reparto, residual_time, tamponi_trasportati, start_node)
print(f"Path: {format_path(path)}")

1. Next node from 'X1' is '7' with time 9.0 and tamponi in path: 6.0. Residual time is 111.0, Tamponi trasportati 6.0. (score=2.2941176470588234)
2. Next node from '7' is '24' with time 5.0 and tamponi in path: 30.0. Residual time is 106.0, Tamponi trasportati 36.0. (score=7.2)
3. Next node from '24' is '22' with time 10.0 and tamponi in path: 3.0. Residual time is 96.0, Tamponi trasportati 39.0. (score=3.3)
4. Next node from '22' is '19' with time 10.0 and tamponi in path: 10.0. Residual time is 86.0, Tamponi trasportati 49.0. (score=1.3)
5. Next node from '19' is '25/2' with time 14.0 and tamponi in path: 4.0. Residual time is 72.0, Tamponi trasportati 53.0. (score=1.0)
6. Next node from '25/2' is '11n' with time 37.0 and tamponi in path: 12.0. Residual time is 35.0, Tamponi trasportati 65.0. (score=0.5945945945945946)
7. Next node from '11n' is '14' with time 10.0 and tamponi in path: 7.0. Residual time is 25.0, Tamponi trasportati 72.0. (score=1.9)
Path: 7 -> 24 -> 22 -> 19 -> 25/2